In [ ]:
#![title](./img/log.png "ShowMyImage")

In [1]:
import sklearn
import pandas as pd
import numpy as np

In [2]:
full_data = pd.read_csv("./data/kc-house-data.csv",index_col=0)
full_data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(full_data,train_size=.8,test_size=.2,random_state=0)

In [8]:
def extract_features(data, features_title):
    features = [data[title].values for title in features_title ]
    #stack them for a 2d [examples,properties]
    return np.stack(features,axis=1)

In [5]:
train_data.columns

Index(['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [9]:
from sklearn.linear_model import LinearRegression

example_features_title = ['sqft_living','bedrooms', 'bathrooms' ]
#extract and stack
example_features = extract_features(train_data,example_features_title)
example_labels=train_data['price']

example_model = LinearRegression().fit(example_features,example_labels)

In [10]:
#Now that we have fitted the model we can extract the regression weights (coefficients) from your model as follows:

example_weight_summary = example_model.coef_
print(example_weight_summary)


[   313.17055038 -56754.66651422   6887.71910816]


In [11]:
example_predictions = example_model.predict(example_features)
example_predictions[0]

395813.49880289414

In [18]:
def get_residual_sum_of_squars(model,data,outcome):
    #first get predictions
    predicted_outcome = model.predict(data)
    #then compute residuals
    RSS = np.sum(np.square(predicted_outcome - outcome))
    return RSS

In [19]:
#Test your function by computing the RSS on TEST data for the example model:

example_test_features = extract_features(test_data,example_features_title)
example_test_labels = test_data['price']
rss_example_test = get_residual_sum_of_squars(example_model,example_test_features,example_test_labels)
print(rss_example_test)

259213572106085.34


In [20]:
from math import log

In [21]:
train_data['bedrooms_squared'] = train_data['bedrooms'].map(lambda cell:cell**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].map(lambda cell:cell**2)

In [22]:
train_data['bed_bath_rooms'] = train_data[['bedrooms', 'bathrooms']].apply(lambda row: row.bedrooms*row.bathrooms, axis=1)
test_data['bed_bath_rooms'] = test_data[['bedrooms', 'bathrooms']].apply(lambda row: row.bedrooms*row.bathrooms, axis=1)
train_data['log_sqft_living'] = train_data['sqft_living'].map(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].map(lambda x: log(x))
train_data['lat_plus_long'] = train_data[['lat', "long"]].agg(sum, axis=1)
test_data['lat_plus_long'] = test_data[['lat', "long"]].agg(sum, axis=1)

C:\Users\gyanr\AppData\Local\Temp\ipykernel_72092\4118047811.py:5: FutureWarning: The provided callable <built-in function sum> is currently using DataFrame.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  train_data['lat_plus_long'] = train_data[['lat', "long"]].agg(sum, axis=1)
C:\Users\gyanr\AppData\Local\Temp\ipykernel_72092\4118047811.py:6: FutureWarning: The provided callable <built-in function sum> is currently using DataFrame.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  test_data['lat_plus_long'] = test_data[['lat', "long"]].agg(sum, axis=1)


In [25]:


# the pandas.Series have a function called exactly that.
print(round(test_data.bedrooms_squared.mean(), 2))
print(round(test_data.bed_bath_rooms.mean(), 2))
print(round(test_data.log_sqft_living.mean(), 2))
print(round(test_data.lat_plus_long.mean(), 2))



12.21
7.45
7.55
-74.65


In [26]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared','log_sqft_living','lat_plus_long']

In [27]:
## Learn the three models. Extract the features and fit your models onto respective features.
# If a goldfish is reading, see the example_model above
features_1 = extract_features(train_data,model_1_features)
features_2 = extract_features(train_data,model_2_features)
features_3 = extract_features(train_data,model_3_features)
output_labels = train_data['price']

model_1=LinearRegression().fit(features_1,output_labels)
model_2=LinearRegression().fit(features_2,output_labels)
model_3=LinearRegression().fit(features_3,output_labels)


In [33]:
print(f"Model-1= {model_1.coef_}, Model-2={model_2.coef_}, Model-3={model_3.coef_}")

Model-1= [ 3.12942010e+02 -5.30962691e+04  1.47770428e+04  6.53983343e+05
 -3.25707336e+05], Model-2=[ 3.06819573e+02 -1.04604718e+05 -7.01815289e+04  6.50590952e+05
 -3.09965751e+05  2.49441497e+04], Model-3=[ 5.37808086e+02  2.78047910e+03  1.01363766e+05  5.30798406e+05
 -4.09655435e+05 -1.81822552e+04  7.24579939e+02 -5.71030023e+05
  1.21142971e+05]


In [34]:
print(f"Model 1 RSS: {get_residual_sum_of_squars(model_1,features_1,output_labels)}")
print(f"Model 2 RSS: {get_residual_sum_of_squars(model_2,features_2,output_labels)}")
print(f"Model 3 RSS: {get_residual_sum_of_squars(model_3,features_3,output_labels)}")

Model 1 RSS: 979843597588329.5
Model 2 RSS: 970799199729577.9
Model 3 RSS: 913653644974958.1


In [35]:


# Compute the RSS on TESTING data for each of the three models and display the values:
features_1_test = extract_features(test_data, model_1_features)
features_2_test = extract_features(test_data, model_2_features)
features_3_test = extract_features(test_data, model_3_features)
output_labels_test = test_data['price']
print("Model 1 RSS: ", get_residual_sum_of_squars(model_1, features_1_test, output_labels_test))
print("Model 2 RSS: ", get_residual_sum_of_squars(model_2, features_2_test, output_labels_test))
print("Model 3 RSS: ", get_residual_sum_of_squars(model_3, features_3_test, output_labels_test))



Model 1 RSS:  213487129319103.6
Model 2 RSS:  210778544168942.5
Model 3 RSS:  203972051917606.94
